In [247]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [248]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [249]:
#全部用平均数填充
numeric = ['Age','RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
for i in numeric:
    train[i].fillna(train[i].mean(), inplace=True)
    test[i].fillna(test[i].mean(), inplace=True)

In [250]:
# 为了定义每列numerical variable，我们将每列column中的每个数都与中位数进行对比，大于中位数为富人（1），其余为穷人（0）

# for column in train[numeric]:
#     for i in range(len(train[column])):
#         if train[column][i] > train[column].median():
#             train[column][i] = 1
#         else:
#             train[column][i] = 0

# for column in test[numeric]:
#     for i in range(len(test[column])):
#         if test[column][i] > test[column].median():
#             test[column][i] = 1
#         else:
#             test[column][i] = 0

In [251]:
#分隔cabin的三个category
train['Cabin'].str.split(pat="/",expand=True)
train['Cabin1'] = train['Cabin'].str.split(pat="/",expand=True)[0]
train['Cabin2'] = train['Cabin'].str.split(pat="/",expand=True)[1]
train['Cabin3'] = train['Cabin'].str.split(pat="/",expand=True)[2]
test['Cabin'].str.split(pat="/",expand=True)
test['Cabin1'] = test['Cabin'].str.split(pat="/",expand=True)[0]
test['Cabin2'] = test['Cabin'].str.split(pat="/",expand=True)[1]
test['Cabin3'] = test['Cabin'].str.split(pat="/",expand=True)[2]

In [252]:
#因为cabin2的分类太多 不予采用 对其他feature进行one hot encoding
data1 = train[['Cabin1', 'Cabin3','HomePlanet','Destination']] 
data2 = test[['Cabin1', 'Cabin3','HomePlanet','Destination']]
data_dummies_1 = pd.get_dummies(data1) 
data_dummies_2 = pd.get_dummies(data2) 
train = train.join(data_dummies_1)
test = test.join(data_dummies_2)
train.drop(["Cabin","Destination","HomePlanet","Name","Cabin1","Cabin2","Cabin3"], inplace = True, axis = 1)
test.drop(["Cabin","Destination","HomePlanet","Name","Cabin1","Cabin2","Cabin3"], inplace = True, axis = 1)

In [253]:
#把passengerid转化成int来使用 不drop
train["PassengerId"] = train["PassengerId"].astype("int")
test["PassengerId"] = test["PassengerId"].astype("int")

# train_c
用随机森林对Cryosleeop进行预测

In [254]:
train_c = train.drop("VIP", axis = 1)

In [255]:
train_c_test = train_c[train_c.isnull().T.any()]
train_c_train = train_c.dropna()

In [256]:
y_train = train_c_train["CryoSleep"].astype('int')

In [257]:
x_train = train_c_train.drop(["CryoSleep"], axis = 1)
x_train.astype('int')

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,101,39,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,201,24,109,9,25,549,44,1,0,0,...,0,0,0,1,1,0,0,0,0,1
2,301,58,43,3576,0,6715,49,0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,302,33,0,1283,371,3329,193,0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,401,16,303,70,151,565,2,1,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,927601,41,0,6819,0,1643,74,0,1,0,...,0,0,1,0,0,1,0,1,0,0
8689,927801,18,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,1,0
8690,927901,26,0,0,1872,1,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
8691,928001,32,0,1049,0,353,3235,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In [258]:
rfc = RandomForestClassifier(random_state = 0) 
rfc.fit(x_train, y_train) 
print("Accuracy on training set: {:.3f}".format(rfc.score(x_train, y_train))) 

Accuracy on training set: 1.000


In [259]:
test_c = train_c_test.drop(["CryoSleep"], axis = 1)
test_c.astype('int')

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
92,9902,2,0,0,0,0,0,1,0,0,...,1,0,1,0,1,0,0,0,0,1
98,10501,27,0,0,570,2,131,0,0,0,...,0,0,1,0,1,0,0,0,0,1
104,11002,40,0,331,0,0,1687,0,0,1,...,0,0,1,0,0,1,0,0,0,1
111,11501,26,0,0,0,0,304,1,0,0,...,0,0,1,0,0,0,1,0,0,1
152,17301,58,0,985,0,5,0,1,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,919701,44,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0
8651,922705,8,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
8664,924601,32,0,0,0,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
8675,925901,44,1030,1015,0,11,304,1,0,0,...,0,0,1,0,1,0,0,0,0,1


In [260]:
y_pred = rfc.predict(test_c) 
y_pred = pd.DataFrame(y_pred)

In [261]:
y_pred.columns = ["CryoSleep"]
y_pred_1 = y_pred.set_index(train_c_test.index)
train_c_test = train_c_test.fillna(y_pred_1)
train_c_test["CryoSleep"] = train_c_test["CryoSleep"].astype("int")
train_c_train["CryoSleep"] = train_c_train["CryoSleep"].astype("int")

/var/folders/6c/32t_r1sd337f9h_jz9wt3hzr0000gn/T/ipykernel_39647/1662174973.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_c_train["CryoSleep"] = train_c_train["CryoSleep"].astype("int")


In [262]:
#把预测的值进行填充
train_c = pd.concat([train_c_train, train_c_test],axis = 0)

In [267]:
train["CryoSleep"] = train_c["CryoSleep"]

# train_vip

In [269]:
train_v = train.copy(deep=True)

In [271]:
train_v_test = train_v[train_v.isnull().T.any()]
train_v_train = train_v.dropna()

In [273]:
y_train_v = train_v_train["VIP"].astype('int')
x_train_v = train_v_train.drop(["VIP"], axis = 1)
x_train_v.astype('int')

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,101,0,39,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
1,201,0,24,109,9,25,549,44,1,0,...,0,0,0,1,1,0,0,0,0,1
2,301,0,58,43,3576,0,6715,49,0,1,...,0,0,0,1,0,1,0,0,0,1
3,302,0,33,0,1283,371,3329,193,0,1,...,0,0,0,1,0,1,0,0,0,1
4,401,0,16,303,70,151,565,2,1,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,927601,0,41,0,6819,0,1643,74,0,1,...,0,0,1,0,0,1,0,1,0,0
8689,927801,1,18,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,1,0
8690,927901,0,26,0,0,1872,1,0,1,0,...,1,0,0,1,1,0,0,0,0,1
8691,928001,0,32,0,1049,0,353,3235,0,0,...,0,0,0,1,0,1,0,1,0,0


In [274]:
rfc = RandomForestClassifier(random_state = 0) 
rfc.fit(x_train_v, y_train_v) 
print("Accuracy on training set: {:.3f}".format(rfc.score(x_train_v, y_train_v))) 

Accuracy on training set: 1.000


In [275]:
#去掉VIP这一列 输入到模型 该列都是空值
test_v = train_v_test.drop(["VIP"], axis = 1)
test_v.astype('int')

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
38,3601,0,15,0,492,48,20,401,0,0,...,0,0,0,1,1,0,0,1,0,0
102,10803,0,0,0,0,0,0,0,1,0,...,1,0,0,1,1,0,0,0,0,1
145,16501,1,35,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,1
228,24402,1,14,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,1,0,0
566,59301,0,28,43,152,182,1,2005,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,907401,1,0,0,0,173,0,0,1,0,...,1,0,0,1,1,0,0,0,0,1
8512,908106,0,16,0,0,761,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
8542,912201,1,55,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,0,0
8630,920503,1,52,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1


In [276]:
y_pred_v = rfc.predict(test_v) 
y_pred_v = pd.DataFrame(y_pred_v)
y_pred_v.columns = ["VIP"]
y_pred_v_1 = y_pred_v.set_index(train_v_test.index)
train_v_test = train_v_test.fillna(y_pred_v_1)
train_v_test["VIP"] = train_v_test["VIP"].astype("int")
train_v_train["VIP"] = train_v_train["VIP"].astype("int")

/var/folders/6c/32t_r1sd337f9h_jz9wt3hzr0000gn/T/ipykernel_39647/1312899910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_v_train["VIP"] = train_v_train["VIP"].astype("int")


In [277]:
train_v = pd.concat([train_v_train, train_v_test],axis = 0)

In [280]:
train["VIP"] = train_v["VIP"]

# test_c

In [284]:
test_c = test.copy(deep=True)
test_c.drop("VIP",inplace = True, axis = 1)

In [286]:
test_c_test = test_c[test_c.isnull().T.any()]
test_c_train = test_c.dropna()

In [288]:
y_train_test_c = test_c_train["CryoSleep"].astype('int')
x_test_c_train = test_c_train.drop(["CryoSleep"], axis = 1)
x_test_c_train.astype('int')

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1301,27,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
1,1801,19,0,9,0,2823,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,1901,31,0,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,1,0,0
3,2101,38,0,6652,0,181,585,0,0,1,...,0,0,0,1,0,1,0,0,0,1
4,2301,20,10,0,635,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,926602,34,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
4273,926901,42,0,847,17,10,144,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4274,927101,28,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4275,927301,28,0,2680,0,0,523,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [289]:
rfc = RandomForestClassifier(random_state = 0) 
rfc.fit(x_test_c_train, y_train_test_c) 
print("Accuracy on training set: {:.3f}".format(rfc.score(x_test_c_train, y_train_test_c))) 

Accuracy on training set: 1.000


In [290]:
test_c_test_test = test_c_test.drop(["CryoSleep"], axis = 1)
test_c_test_test.astype('int')

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
68,15601,28,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
78,17502,8,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,1,0,0
92,20801,40,44,0,565,0,18,0,0,0,...,1,0,1,0,1,0,0,0,0,1
99,22701,22,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
165,34901,33,0,1030,0,3490,5832,1,0,0,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,870501,20,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4108,892506,27,0,2182,7,2582,19,0,0,1,...,0,0,1,0,0,1,0,0,0,1
4209,913801,41,0,1998,0,1023,867,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4230,918201,14,0,0,177,1377,29,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [291]:
y_pred_test_c = rfc.predict(test_c_test_test) 
y_pred_test_c = pd.DataFrame(y_pred_test_c)
y_pred_test_c.columns = ["CryoSleep"]
y_pred_test_c_1 = y_pred_test_c.set_index(test_c_test.index)
test_c_test = test_c_test.fillna(y_pred_test_c_1)
test_c_test["CryoSleep"] = test_c_test["CryoSleep"].astype("int")
test_c_train["CryoSleep"] = test_c_train["CryoSleep"].astype("int")

/var/folders/6c/32t_r1sd337f9h_jz9wt3hzr0000gn/T/ipykernel_39647/4278463944.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_c_train["CryoSleep"] = test_c_train["CryoSleep"].astype("int")


In [292]:
test_c = pd.concat([test_c_train, test_c_test],axis = 0)

In [295]:
test["CryoSleep"] = test_c["CryoSleep"]

# test_vip

In [298]:
testset_v = test.copy(deep=True)
testset_v_test = testset_v[testset_v.isnull().T.any()]
testset_v_train = testset_v.dropna()

In [299]:
y_train_testset_v = testset_v_train["VIP"].astype('int')
x_train_testset_v = testset_v_train.drop(["VIP"], axis = 1)
x_train_testset_v.astype('int')

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1301,1,27,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
1,1801,0,19,0,9,0,2823,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,1901,1,31,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0
3,2101,0,38,0,6652,0,181,585,0,0,...,0,0,0,1,0,1,0,0,0,1
4,2301,0,20,10,0,635,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,926602,1,34,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
4273,926901,0,42,0,847,17,10,144,0,0,...,0,0,0,0,1,0,0,0,0,1
4274,927101,1,28,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4275,927301,0,28,0,2680,0,0,523,0,0,...,0,0,1,0,0,1,0,0,0,0


In [300]:
rfc = RandomForestClassifier(random_state = 0) 
rfc.fit(x_train_testset_v, y_train_testset_v) 
print("Accuracy on training set: {:.3f}".format(rfc.score(x_train_testset_v, y_train_testset_v))) 

Accuracy on training set: 1.000


In [301]:
v_test = testset_v_test.drop(["VIP"], axis = 1)
v_test.astype('int')

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
42,10001,0,28,0,17,997,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
93,20901,0,36,33,439,0,12767,1193,0,1,...,0,0,1,0,0,1,0,0,0,1
101,22802,0,39,0,25273,0,13,1714,0,1,...,0,0,0,1,0,1,0,1,0,0
148,32501,0,48,219,755,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
150,32901,1,33,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,887502,0,32,26,1708,0,3993,481,0,0,...,0,0,0,1,0,1,0,0,0,1
4087,889501,1,39,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4176,905801,0,38,784,0,42,38,10,0,0,...,0,0,0,0,0,0,1,0,0,1
4250,922801,0,21,640,0,21,0,76,0,0,...,0,0,1,0,1,0,0,0,0,1


In [302]:
y_pre = rfc.predict(v_test) 
y_pre = pd.DataFrame(y_pre)
y_pre.columns = ["VIP"]
y_pre_1 = y_pre.set_index(testset_v_test.index)
testset_v_test = testset_v_test.fillna(y_pre_1)
testset_v_test["VIP"] = testset_v_test["VIP"].astype("int")
testset_v_train["VIP"] = testset_v_train["VIP"].astype("int")

/var/folders/6c/32t_r1sd337f9h_jz9wt3hzr0000gn/T/ipykernel_39647/3592643918.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testset_v_train["VIP"] = testset_v_train["VIP"].astype("int")


In [303]:
testset_v = pd.concat([testset_v_train, testset_v_test],axis = 0)

In [304]:
test["VIP"] = testset_v["VIP"]

训练模型 产出结果

In [308]:
#对模型进行训练
Y_train = train["Transported"]
X_train = train.drop("Transported", axis = 1)
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, Y_train)
Y_pred = rfc.predict(test)
Y_pred = pd.DataFrame(Y_pred)

In [313]:
#训练完模型后的结果导出
# test["CryoSleep"] = test["CryoSleep"].astype('bool')
# test["VIP"] = test["VIP"].astype('bool')
# test = test.astype("int")
test_1 = pd.read_csv("test.csv")
Y_pred["PassengerId"] = test_1["PassengerId"]
Y_pred["PassengerId"] = test_1["PassengerId"]
Y_pred["Transported"] = Y_pred[0]
#y_pred["Transported"] = y_pred["Transported"].astype("bool") 视模型类型决定需不需要重新转data type
Y_pred.drop(columns = 0, inplace = True)
Y_pred.to_csv("./spaceship.csv",  index = None)